In [34]:
import cvxpy
from matplotlib import pyplot as plt
import numpy as np
from src.components.microgrid_simple import SimpleMicrogrid
from src.components.battery import Battery, BatteryParameters
from src.rl.a2c.c_mg_simple import set_all_seeds

In [35]:
set_all_seeds(0)

### Set env vars

In [41]:
n = 24

mg = SimpleMicrogrid(steps=n)
# base_demand, demand = mg.demand_family(min_noise=0, max_noise=0)
base_demand, demand = mg.demand_home_business(min_noise=0, max_noise=0)
base_pv, pv = mg.pv_generation(min_noise=0, max_noise=0)
_, _, _, price_s, emission = mg.grid_price_and_emission()

price_b = price_s / 4

In [42]:
battery_params = {
    "soc_0": 0.1,
    "soc_max":0.9,
    "soc_min":0.1,
    "p_charge_max":0.8,
    "p_discharge_max":0.8,
    "efficiency":0.9,
    "capacity":1,
    "sell_price":0.0,
    "buy_price":0.0
}

real_battery = Battery(random_soc_0=False, params = BatteryParameters(battery_params))

p_charge, p_discharge, _ = real_battery.check_battery_constraints(power_rate=0.8)
real_battery.apply_action(p_charge=p_charge, p_discharge=p_discharge)

real_battery.soc.item()

# battery.reset()

0.7200000000000001

### Adv battery calc (not working)

In [57]:
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery

constraints.append(battery[0] == battery_params["soc_min"])

for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't

for i in range(n):
    constraints.append(action[i] <= 1)
    constraints.append(action[i] >= -1)

# Transition

obj = 0

for i in range(n):

    # Get the battery constraints

    # p_charge, p_discharge, _ = real_battery.check_battery_constraints(power_rate=action[i])

    # real_battery.apply_action(p_charge=p_charge, p_discharge=p_discharge)

    # p_charge = p_charge.item()
    # p_discharge = p_discharge.item()

    capacity_to_charge = cvxpy.maximum(
        (battery_params["soc_max"] * battery_params["capacity"] - battery[i] * battery_params["capacity"]) / battery_params["efficiency"],
        0
    )

    capacity_to_discharge = cvxpy.maximum(
        (battery[i] * battery_params["capacity"] - battery_params["soc_min"] * battery_params["capacity"]) / battery_params["efficiency"],
        0
    )

    # p_charge = cvxpy.maximum(action[i], 0)
    # p_discharge = cvxpy.maximum(-action[i], 0)
    p_charge = 0
    p_discharge = 0
    
    constraints.append(p_charge <= capacity_to_charge)
    constraints.append(p_discharge <= capacity_to_discharge)

    constraints.append(consumption[i] == base_demand[i]-base_pv[i] + p_charge + p_discharge)
    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

DCPError: Problem does not follow DCP rules. Specifically:
The following constraints are not DCP:
0.0 <= maximum((0.9 + -var62552[0] @ 1.0) / 0.9, 0.0) , because the following subexpressions are not:
|--  0.0 <= maximum((0.9 + -var62552[0] @ 1.0) / 0.9, 0.0)
0.0 <= maximum((var62552[0] @ 1.0 + -0.1) / 0.9, 0.0) , because the following subexpressions are not:
|--  0.0 <= maximum((var62552[0] @ 1.0 + -0.1) / 0.9, 0.0)
0.0 <= maximum((0.9 + -var62552[1] @ 1.0) / 0.9, 0.0) , because the following subexpressions are not:
|--  0.0 <= maximum((0.9 + -var62552[1] @ 1.0) / 0.9, 0.0)
0.0 <= maximum((var62552[1] @ 1.0 + -0.1) / 0.9, 0.0) , because the following subexpressions are not:
|--  0.0 <= maximum((var62552[1] @ 1.0 + -0.1) / 0.9, 0.0)
0.0 <= maximum((0.9 + -var62552[2] @ 1.0) / 0.9, 0.0) , because the following subexpressions are not:
|--  0.0 <= maximum((0.9 + -var62552[2] @ 1.0) / 0.9, 0.0)
0.0 <= maximum((var62552[2] @ 1.0 + -0.1) / 0.9, 0.0) , because the following subexpressions are 

### Real dataset

In [47]:
battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)

constraints = []
# Battery

constraints.append(battery[0] == battery_params["soc_min"])

for i in range(n+1):
    constraints.append(battery[i] <= battery_params["soc_max"])
    constraints.append(battery[i] >= battery_params["soc_min"])


# Action / Batteryn't

for i in range(n):
    constraints.append(action[i] <= 1)
    constraints.append(action[i] >= -1)


# Transition

obj = 0

for i in range(n):
    
    constraints.append(action[i] <= battery_params["p_charge_max"])
    constraints.append(action[i] <= battery_params["p_discharge_max"])
    constraints.append(battery[i+1] == battery[i] + action[i])

    constraints.append(consumption[i] == demand[i]-pv[i] + action[i])


    obj += cvxpy.maximum(consumption[i] * (price_s[i] + emission[i]),0) 
    obj += cvxpy.maximum(-consumption[i] * price_b[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

(0.855431155773072,
 array([0.1       , 0.40852198, 0.47973789, 0.50411047, 0.53223491,
        0.61301837, 0.9       , 0.8582515 , 0.22036464, 0.1       ,
        0.24436358, 0.36342561, 0.51372866, 0.66327171, 0.78990254,
        0.9       , 0.70843913, 0.26658067, 0.1       , 0.40107527,
        0.46417861, 0.46771514, 0.47094213, 0.332708  , 0.1       ]),
 array([ 0.30852198,  0.0712159 ,  0.02437259,  0.02812444,  0.08078347,
         0.28698163, -0.0417485 , -0.63788686, -0.12036464,  0.14436358,
         0.11906203,  0.15030305,  0.14954305,  0.12663083,  0.11009746,
        -0.19156087, -0.44185845, -0.16658067,  0.30107527,  0.06310335,
         0.00353652,  0.00322699, -0.13823413, -0.232708  ]))

### Test Dataset

In [ ]:
n = 4
d = np.random.rand(n)
g = np.random.rand(n)*0.5
pb = np.random.rand(n)
ps = pb/4

In [ ]:

battery = cvxpy.Variable(n+1)
action = cvxpy.Variable(n)
consumption = cvxpy.Variable(n)


constraints = []
for i in range(n+1):
    constraints.append(battery[i] <= 1)
    constraints.append(battery[i] >= 0)
constraints.append(battery[0] == 0)

for i in range(n):
    constraints.append(action[i] <= .3)
    constraints.append(action[i] >= -1)
    

obj = 0
for i in range(n):
    constraints.append(battery[i+1] == battery[i] + action[i]  )

    constraints.append(consumption[i] == d[i]-g[i] + action[i] )


    obj += cvxpy.maximum(consumption[i]* pb[i],0) 
    obj += cvxpy.maximum(-consumption[i]*ps[i],0)  


objective = cvxpy.Minimize(obj)
prob = cvxpy.Problem(objective, constraints)
res = prob.solve()

res, battery.value, action.value

(0.769357835608137,
 array([1.13622974e-14, 3.00000000e-01, 6.00000000e-01, 1.14580415e-01,
        1.65161251e-12]),
 array([ 0.3       ,  0.3       , -0.48541958, -0.11458041]))

In [ ]:


x_hat = cvxpy.Variable(A.shape[1])
objective = cvxpy.Minimize(cvxpy.norm(x_hat, 1))
    constraints = [A * x_hat == y]
    prob = cvxpy.Problem(objective, constraints)
    prob.solve()
    return np.squeeze(np.array(x_hat.value)), (prob.value, prob.status)